   # Hello PyTorch with MLflow

In this example, we like to demonstrate that the example code used in hello-pt-tb with PyTorch Tensorboard tracking can be simply switched to using an MLflow tracking server without changing the code.



Example of using [NVIDIA FLARE](https://nvflare.readthedocs.io/en/main/index.html) to train an image classifier using federated averaging ([FedAvg](https://arxiv.org/abs/1602.05629)) and [PyTorch](https://pytorch.org/) as the deep learning training framework. This example also highlights the streaming capability from the clients to the server with Tensorboard SummaryWriter sender syntax, but with a MLflow receiver

> **_NOTE:_** This example uses the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset and will load its data within the trainer code.


### 1. Install NVIDIA FLARE

Follow the [Installation](https://nvflare.readthedocs.io/en/main/getting_started.html#installation) instructions.
Install additional requirements (if you already have a specific version of nvflare installed in your environment, you may want to remove nvflare in the requirements to avoid reinstalling nvflare):


In [ ]:
%pip install torch torchvision tensorboard mlflow

### 2. Change Configuration

in fed_server_config.json

add the following to the components
```
{
      "id": "mlflow_receiver",
      "path": "nvflare.app_opt.tracking.mlflow.mlflow_receiver.MLflowReceiver",
      "args": {
        "kw_args": {"experiment_name": "hello-pt-experiment"},
        "artifact_location": "artifacts"
      }
}
```
This indicates that we are registering the MLflow Receiver in additional to the Tensorboard Receiver.

Note that the job hello-pt-mlflow is an example using mlflow syntax and the MLflowWriter on the client side, and
hello-pt-tb-mlflow has the learner using tb syntax. Both work with MLflowReceiver.



### 3. Run the experiment

Use nvflare simulator to run the examples with the additional common python files included in the python path:

export PYTHONPATH=${PWD}/..

```
nvflare simulator -w /tmp/nvflare/ -n 2 -t 2 ./jobs/hello-pt-tb-mlflow
```

or set the PYTHONPATH programmatically. 


In [ ]:
import os
parent_directory = os.path.dirname(os.getcwd())

# Get the current PATH
current_path = os.environ.get('PYTHONPATH', '')

# Add the path if it's not already there
if parent_directory not in current_path:
    os.environ['PYTHONPATH'] = parent_directory + os.pathsep + current_path


In [ ]:
!nvflare simulator -w /tmp/nvflare/ -n 2 -t 2 ./jobs/hello-pt-tb-mlflow

### 4. Tensorboard Tracking

On the client side, we are still using the TensorBoard SummaryWriter as the `AnalyticsSender`. 

Instead of writing to TB files, it actually generates NVFLARE events of type `analytix_log_stats`.
The `ConvertToFedEvent` widget will turn the event `analytix_log_stats` into a fed event `fed.analytix_log_stats`,
which will be delivered to the server side.

On the server side, the `TBAnalyticsReceiver` is configured to process `fed.analytix_log_stats` events,
which writes received TB data into appropriate TB files on the server.

To view training metrics that are being streamed to the server, run:

```
tensorboard --logdir=/tmp/nvflare/server/simulate_job/tb_events
```

In [ ]:
!tensorboard --logdir=/tmp/nvflare/server/simulate_job/tb_events

>Note 
Remember to "stop" above cell before running next cell

### 5. MLflow tracking

On the server side, we also configured `MLflowReceiver` to process `fed.analytix_log_stats` events,
which writes received events to the MLflow tracking server.

To view training metrics that are being streamed to the server, run:

```
mlflow ui --backend-store-uri=/tmp/nvflare/server/simulate_job/mlruns
```

Then 

Look at the URL in browser http://localhost:5000/

In [ ]:
!mlflow ui --backend-store-uri=/tmp/nvflare/server/simulate_job/mlruns

> Note: remember to "stop" above cell